# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [2]:
env = UnityEnvironment(file_name='./Reacher_x20_Linux/Reacher.x86_64')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_size -> 5.0
		goal_speed -> 1.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726624e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [5]:
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

Total score (averaged over agents) this episode: 0.11599999740719795


When finished, you can close the environment.

In [6]:
# env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

### Create an environment wrapper to make it more operable

In [7]:
class EnvWrapper:

    def __init__(self, env, brain_name: str) -> None:
        self.env = env
        self.brain_name = brain_name
        self.brain = env.brains[brain_name]
        self.action_size = self.brain.vector_action_space_size
        self.state_size = self.reset(False).shape
    
    def reset(self, train_mode: bool):
        env_info = self.env.reset(train_mode=train_mode)[self.brain_name]
        return env_info.vector_observations

    def step(self, actions):
        env_info = self.env.step(actions)[self.brain_name]
        next_states = env_info.vector_observations
        rewards = env_info.rewards
        dones = env_info.local_done
        return next_states, rewards, dones


    def get_state_size(self):
        return self.state_size

    def get_action_size(self):
        return self.action_size

### Define Models (Actor and Critic Networks)

In [8]:
from typing import Tuple
import torch
import torch.nn as nn
import torch.nn.functional as F


def hidden_init(layer) -> Tuple[float, float]:
    fan_in = layer.weight.data.size()[0]
    lim = 1. / np.sqrt(fan_in)
    return (-lim, lim)

class Actor(nn.Module):
    def __init__(
        self, state_size: int, 
        action_size: int, 
        seed: int, 
        fc1_units: int=128, 
        fc2_units: int=128,
        fc3_units: int=128) -> None:
        super().__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.bn1 = nn.BatchNorm1d(fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        # self.bn2 = nn.BatchNorm1d(fc2_units)
        # self.fc3 = nn.Linear(fc2_units, fc3_units)
        self.fc3 = nn.Linear(fc2_units, action_size)
        # self.bn3 = nn.BatchNorm1d(fc3_units)
        # self.fc4 = nn.Linear(fc3_units, action_size)
        self.reset_parameters()

    def reset_parameters(self) -> None:
        self.fc1.weight.data.uniform_(*hidden_init(self.fc1))
        self.fc2.weight.data.uniform_(*hidden_init(self.fc2))
        # self.fc3.weight.data.uniform_(*hidden_init(self.fc3))
        self.fc3.weight.data.uniform_(-3e-3, 3e-3)

    def forward(self, state):
        x = torch.relu(self.fc1(state))
        x = self.bn1(x)
        x = torch.relu(self.fc2(x))
        # x = torch.relu(self.fc3(x))
        return torch.tanh(self.fc3(x))



class Critic(nn.Module):
    def __init__(
        self, 
        state_size: int, 
        action_size: int, 
        seed: int, 
        fc1_units: int=128, 
        fc2_units: int=128,
        fc3_units: int=128) -> None:
        super().__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.bn1 = nn.BatchNorm1d(fc1_units)
        self.fc2 = nn.Linear(fc1_units + action_size, fc2_units)
        # self.bn2 = nn.BatchNorm1d(fc2_units)
        # self.fc3 = nn.Linear(fc2_units, fc3_units)
        # self.bn3 = nn.BatchNorm1d(fc3_units)
        self.fc3 = nn.Linear(fc2_units, 1)
        self.reset_parameters()

    def reset_parameters(self) -> None:
        self.fc1.weight.data.uniform_(*hidden_init(self.fc1))
        self.fc2.weight.data.uniform_(*hidden_init(self.fc2))
        # self.fc3.weight.data.uniform_(*hidden_init(self.fc3))
        self.fc3.weight.data.uniform_(-3e-3, 3e-3)

    def forward(self, state, action):
        xs = torch.relu(self.fc1(state))
        xs = self.bn1(xs)
        x = torch.cat((xs, action), dim=1)
        x = torch.relu(self.fc2(x))
        # x = torch.relu(self.fc3(x))
        return self.fc3(x)



### Define Memory buffer

As part of the DQN algorithm, we need to use the experience replay buffer. Hence the class below defines the minimal logic for a experience replay buffer.


In [9]:
from collections import namedtuple, deque
import random


class ReplayBuffer:
    """Replay buffer.
    
    This class implements a simple replay
    buffer of a fixed size and able to
    take random samples of a fixed size.
    """
    def __init__(self, buffer_size, batch_size, seed, device):
        self.batch_size = batch_size
        self.seed = random.seed(seed)
        self.device = device
        
        self.memory = deque(maxlen=buffer_size)
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
    
    def add(self, state, action, reward, next_state, done):
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
        
    def sample(self):
        """Sample.
        
        Selects randomly a batch of sise batch_size
        from the current memory buffer. It delivers a
        tuple of (states, actions, next_states, dones)
        every element as a torch tensor.
        """
        experiences = random.sample(self.memory, k=self.batch_size)
        
        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(self.device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(self.device)
        n_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(self.device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(self.device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(self.device)
        
        return (states, actions, n_states, rewards, dones)
    
    def __len__(self):
        return len(self.memory)

In [10]:
import copy 

class OUNoise:
    def __init__(self, size: int, seed: int, mu: int=0, theta: float=0.15, sigma: float=0.2) -> None:
        self.mu = mu * np.ones(size)
        self.theta = theta
        self.sigma = sigma
        self.seed = random.seed(seed)
        self.reset()

    def reset(self):
        self.state = copy.copy(self.mu)

    def sample(self):
        x = self.state
        dx = self.theta * (self.mu - x) + self.sigma * np.array([random.random() for i in range(len(x))])
        self.state = x + dx
        return self.state

### Define the Agent
The agent will make use of both networks, one for taking action and the other for evaluation.

In [11]:
import torch.optim as optim
# from torch.utils.tensorboard import SummaryWriter

class Agent:
    def __init__(
        self, 
        state_size, 
        action_size, 
        random_seed,
        gamma: float=0.99,
        tau: float=1e-3,
        lr_actor: float=1e-4,
        lr_critic: float=1e-3, 
        weight_decay: float=0.0,
        buffer_size: int=int(1e6),
        batch_size: int=128,
        update_every: int=20,
        device: str="cpu",
        noise = None,
        # writer :SummaryWriter = None
        ) -> None:
        self.state_size = state_size
        self.action_size = action_size
        self.seed = random.seed(random_seed)
        self.device = device

        self.batch_size = batch_size
        self.gamma = gamma
        self.tau = tau
        self.update_every = update_every
        self._t_step = 0

        self.actor_local = Actor(state_size, action_size, random_seed).to(device)
        self.actor_target = Actor(state_size, action_size, random_seed).to(device)
        self.actor_optimizer = optim.Adam(self.actor_local.parameters(), lr=lr_actor)

        self.critic_local = Critic(state_size, action_size, random_seed).to(device)
        self.critic_target = Critic(state_size, action_size, random_seed).to(device)
        self.critic_optimizer = optim.Adam(self.critic_local.parameters(), lr=lr_critic, weight_decay=weight_decay)

        self.clone_weights(self.actor_target, self.actor_local)
        self.clone_weights(self.critic_target, self.critic_local)

        self.noise = OUNoise(action_size, random_seed) if not noise else noise

        self.memory = ReplayBuffer(action_size, buffer_size, batch_size, device)

        # self.writer = writer


    def step(self, state, action, reward, next_state, done):
        self.memory.add(state, action, reward, next_state, done)

        if self._t_step % self.update_every == 0:
            if len(self.memory) > self.batch_size:
                experiences = self.memory.sample()
                self.learn(experiences, self.gamma)
        self._t_step += 1

    def act(self, state, add_noise=True, eps=1.0):
        state = torch.from_numpy(state).float().to(self.device)
        self.actor_local.eval()
        
        with torch.no_grad():
            action = self.actor_local(state).cpu().data.numpy()
        
        self.actor_local.train()
        
        if add_noise:
            action += (self.noise.sample() * eps)
        
        return np.clip(action, -1, 1)

    def clone_weights(self, w1, w0):
        for p1, p0 in zip(w1.parameters(), w0.parameters()):
            p1.data.copy_(p0.data)

    def reset(self):
        self.noise.reset()

    def learn(self, experiences, gamma):
        states, actions, rewards, next_states, dones = experiences

        actions_next = self.actor_target(next_states)
        q_targets_next = self.critic_target(next_states, actions_next)

        q_targets = rewards + (gamma * q_targets_next * (1 - dones))

        q_expected = self.critic_local(states, actions)
        critic_loss = F.mse_loss(q_expected, q_targets)
        # if self.writer is not None:
        #     self.writer.add_scalar("critic_loss", critic_loss.item(), self._t_step)

        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        torch.nn.utils.clip_grad_norm_(self.critic_local.parameters(), 1)
        self.critic_optimizer.step()


        actions_pred = self.actor_local(states)
        actor_loss = -self.critic_local(states, actions_pred).mean()
        # if self.writer is not None:
        #     self.writer.add_scalar("actor_loss", actor_loss.item())
        
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()


        self.soft_update(self.critic_local, self.critic_target, self.tau)
        self.soft_update(self.actor_local, self.actor_target, self.tau)

    def soft_update(self, local_model, target_model, tau):
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_model.data.copy_(tau * local_param.data + (1.0 - tau) * target_param.data)



### DDPG Algorithm


In [14]:
from tqdm import tqdm

def ddpg(
    agent: Agent,
    env: EnvWrapper,
    num_agents: int=20,
    n_episodes: int=1000,
    max_t: int=300,
    score_window: int=100,
    eps_start: float=1.0,
    eps_end: float=0.1,
    eps_decay: float=1e-6,
    add_action_noise: bool=True
):
    scores_deque = deque(maxlen=score_window)
    scores = []
    eps = eps_start
    with tqdm(total=n_episodes) as progress:
        for i_episode in range(1, n_episodes + 1):
            states = env.reset(train_mode=True)
            agent.reset()
            score = np.zeros(num_agents)
            for t in range(max_t):
                if t % 100:
                    agent.reset()
                
                actions = agent.act(states, add_noise=add_action_noise, eps=eps)
                next_states, rewards, dones = env.step(actions)

                # for i in range(num_agents):
                #     agent.step(states[i], actions[i], rewards[i], next_states[i], dones[i])
                agent.step(states, actions, rewards, next_states, dones)

                states = next_states
                score += rewards
                # eps = eps - eps_decay
                # eps = max(eps, eps_end)
                if np.any(dones):
                    break
            
            
            
            score = np.mean(score)
            scores_deque.append(score)
            scores.append(score)
            progress.set_postfix({"Avg. Score": f"{np.mean(scores_deque):.2f}"})
            progress.update()

            if np.mean(scores_deque) >= 30.0:
                print(f"Environment solved at {i_episode} episodes with Avg. score: {np.mean(scores_deque):.2f}")
                torch.save(agent.actor_local.state_dict(), 'cp_actor.pth')
                torch.save(agent.critic_local.state_dict(), 'cp_critic.pth')
                break
    return scores


In [15]:
%%time
# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"
env_wrapper = EnvWrapper(env, brain_name)
state = env_wrapper.reset(True)
state_size = env_wrapper.get_state_size()[1]
action_size = env_wrapper.get_action_size()
random_seed = 123
# writer = SummaryWriter(log_dir="./tensorboard/")

noise = OUNoise(action_size, random_seed, theta=0.15, sigma=0.1)
agent = Agent(
    state_size, 
    action_size, 
    random_seed, 
    device=device,
    buffer_size=int(1e6),
    batch_size=128,
    tau=1e-3, 
    lr_actor=2e-4, 
    lr_critic=2e-3, 
    gamma=0.99,
    update_every=1,
    noise=noise)

scores = ddpg(agent, env_wrapper, n_episodes=200, max_t=1000, add_action_noise=True)

 54%|█████▎    | 107/200 [09:01<08:14,  5.32s/it, Avg. Score=0.02]

In [ ]:
import matplotlib.pyplot as plt

def plot_scores(scores):
    """plot scores."""
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111)
    
    x = np.arange(len(scores))
    y = scores
    
    plt.plot(x, y, label="avg. scores")
    
    plt.ylabel("Score")
    plt.xlabel("Episode #")
    plt.title("Agent progress over episodes")
    plt.show()